In [1]:
import pandas as pd
from pathlib import Path
import os

In [2]:
notebook_path = Path(os.getcwd())
repo_path = notebook_path.parent.absolute()
root_path = repo_path.parent.absolute()
data_path = os.path.join(root_path, 'data')
os.chdir(root_path)